In [1]:
import os

from models import multiscale_model as multiscale
from models import model
from models import track_model
from src.utils import visualize_detection_video, visualize_tracking_video

In [2]:
videos_folder = "data/test_data/det_track/RGB/videos"
model_path = "checkpoints/RGB/yolov8n2/best.pt"
output_folder = "outputs/tunning"

In [3]:
videos_path = [
    os.path.join(videos_folder, video)
    for video in os.listdir(videos_folder)
    if video.endswith(('.mp4', '.avi', '.mov'))
]

In [4]:
videos_path

['data/test_data/det_track/RGB/videos\\V_BIRD_043.mp4',
 'data/test_data/det_track/RGB/videos\\V_BIRD_045.mp4',
 'data/test_data/det_track/RGB/videos\\V_BIRD_047.mp4',
 'data/test_data/det_track/RGB/videos\\V_BIRD_050.mp4',
 'data/test_data/det_track/RGB/videos\\V_BIRD_051.mp4',
 'data/test_data/det_track/RGB/videos\\V_DRONE_099.mp4',
 'data/test_data/det_track/RGB/videos\\V_DRONE_101.mp4',
 'data/test_data/det_track/RGB/videos\\V_DRONE_104.mp4',
 'data/test_data/det_track/RGB/videos\\V_DRONE_105.mp4',
 'data/test_data/det_track/RGB/videos\\V_DRONE_108.mp4',
 'data/test_data/det_track/RGB/videos\\V_DRONE_111.mp4',
 'data/test_data/det_track/RGB/videos\\V_DRONE_114.mp4']

In [5]:
multiscale_model = multiscale.DetectionModel(
                model_path=model_path,
                device="cuda",
            )
yolo_model = model.DetectionModel(
                model_path=model_path,
                device="cuda",
            )
track_model = track_model.TrackingModel(
                multiscale_model
            )

In [ ]:
for iou in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
    for conf in [0.3, 0.4, 0.5, 0.6]:
        print(f"Running with iou: {iou}, conf: {conf}")
        for path in videos_path:
            video_name = os.path.splitext(os.path.basename(path))[0]
            print(f"Processing video: {video_name}")
            output_path = f"{output_folder}/iou_{iou}_conf_{conf}/{video_name}"
            
            multiscale_det = multiscale_model.video_detect(
                                                    path,
                                                    conf_threshold=conf,
                                                    iou_threshold=iou    
                                                )
            yolo_det = yolo_model.video_detect(
                                            path,
                                            conf_threshold=conf,
                                            iou_threshold=iou
                                        )
                
            visualize_detection_video(
                video_path=path,
                detection_frames=yolo_det,
                output_path=f"{output_path}/yolo.mp4"
            )
            visualize_detection_video(
                video_path=path,
                detection_frames=multiscale_det,
                output_path=f"{output_path}/multiscale.mp4"
            )
            
            with open(f"{output_path}/yolo_det.txt", "w") as f:
                for idx, frame in enumerate(yolo_det):
                    f.write(f"Frame {idx}:\n")
                    for box, score, label in zip(frame['boxes'], frame['scores'], frame['labels']):
                        box = [float(coord) for coord in box]
                        label = int(label)
                        score = float(score)
                        f.write(f"Box: {box}, Score: {score}, Label: {label}\n")
            
            with open(f"{output_path}/multiscale_det.txt", "w") as f:
                for idx, frame in enumerate(multiscale_det):
                    f.write(f"Frame {idx}:\n")
                    for box, score, label in zip(frame['boxes'], frame['scores'], frame['labels']):
                        box = [float(coord) for coord in box]
                        label = int(label)
                        score = float(score)
                        f.write(f"Box: {box}, Score: {score}, Label: {label}\n")

Running with iou: 0.1, conf: 0.3
Processing video: V_BIRD_043
Processing video: V_BIRD_045
Processing video: V_BIRD_047
Processing video: V_BIRD_050
Processing video: V_BIRD_051
Processing video: V_DRONE_099


In [ ]:
# yolo_model = model.DetectionModel(
#                             model_path,
#                             conf_threshold=0.3,
#                             iou_threshold=0.45
#                             )
# multiscale_model = multiscale.DetectionModel(
#                                     model_path,
#                                     conf_threshold=0.3,
#                                     iou_threshold=0.1
#                                     )
# yolo_track_model = track_model.TrackingModel(
#                             yolo_model,
#                             )
# multiscale_track_model = track_model.TrackingModel(
#                             multiscale_model,
#                             )

In [ ]:
# for path in videos_path:
#     video_name = os.path.basename(path).removesuffix('.mp4').removesuffix('.avi').removesuffix('.mov')
#     print(f"Processing video: {video_name}")
    
#     # # Detection
#     yolo_det = yolo_model.video_detect(path)
#     multiscale_det = multiscale_model.video_detect(path)
    
#     visualize_detection_video(
#         video_path=path,
#         detection_frames=yolo_det,
#         output_path=f"{output_folder}/detection/{video_name}/yolo.mp4"
#     )
#     visualize_detection_video(
#         video_path=path,
#         detection_frames=multiscale_det,
#         output_path=f"{output_folder}/detection/{video_name}/multiscale.mp4"
#     )
    
#     # # Tracking
#     yolo_track = yolo_track_model.video_track(path)
#     multiscale_track = multiscale_track_model.video_track(path)
#     visualize_tracking_video(
#         video_path=path,
#         tracking_frames=yolo_track,
#         output_path=f"{output_folder}/tracking/{video_name}/yolo.mp4"
#     )
#     visualize_tracking_video(
#         video_path=path,
#         tracking_frames=multiscale_track,
#         output_path=f"{output_folder}/tracking/{video_name}/multiscale.mp4"
#     )